In [ ]:
# Chargement des librairies nécessaires
library(ggplot2)
library(tidyverse)
library(gridExtra)
library(GGally)
library(plotly)
library(corrplot)
library(reshape2)
library(FactoMineR) 
library(factoextra)
library(glmnet) 
library(ggfortify)
library(pROC)
library(ROCR)
library(repr)
library(caret)

# fix random seed for reproducibility
set.seed(1234)

## GRAPH SETTINGS ##
# Save original parameters (optional)
original_par <- par(no.readonly = TRUE)

# Set global scaling factors (1.5x default size)
par(
  cex.lab = 1.5,   # Axis labels
  cex.axis = 1.5,  # Axis text (tick labels)
  cex.main = 1.5,  # Main title
  cex.sub = 1.5    # Subtitle
)

# Define a custom theme with larger fonts
custom_theme <- theme(
  text = element_text(size = 16),            # Global text size
  axis.title = element_text(size = 18),      # Axis labels
  axis.text = element_text(size = 14),       # Axis tick labels
  plot.title = element_text(size = 20),      # Main title
  plot.subtitle = element_text(size = 16)    # Subtitle
)

# Apply the theme to all future plots
theme_set(custom_theme)


## DATA LOADING & PROCESSING ##
# Load data
path <- "../../" # modifier le nombre de ../ si nécessaire
gym <- read.table(paste(path, "gym_members_exercise_tracking.csv", sep = ""),
                    sep = ",", header = TRUE)

gym[,'Gender'] <- as.factor(gym[,'Gender'])
gym[,'Workout_Type'] <- as.factor(gym[,'Workout_Type'])
gym[,'Experience_Level'] <- as.factor(gym[,'Experience_Level'])
gym[,'Workout_Frequency..days.week.'] <- as.factor(gym[,'Workout_Frequency..days.week.'])

gym[, "Weight..kg."] <- log(gym[,"Weight..kg."])

max_fat <- max(gym[,"Fat_Percentage"])
gym[, "Fat_Percentage"] <- sqrt((max_fat + 1) - gym[,"Fat_Percentage"])

# renome les variables Weight..kg. et BMI en LWeight et LBMI
names(gym)[names(gym) == "Weight..kg."] <- "LWeight"
names(gym)[names(gym) == "Fat_Percentage"] <- "SFat_Percentage"

gym <- gym %>% select(-c(BMI))

# divide data into training and testing sets for experience level
trainIndex <- createDataPartition(gym$Experience_Level, p = .8, 
                                  list = FALSE, 
                                  times = 1)
gym_train <- gym[ trainIndex,]
gym_test  <- gym[-trainIndex,]

# Normalize the data
gym_train_scaled = gym_train
scaler <- scale(gym_train[,-c(2,10,13,14)])

# Extract the center and scale attributes
center <- attr(scaler, "scaled:center")
scale <- attr(scaler, "scaled:scale")

gym_train_scaled[,-c(2,10,13,14)] <- scale(gym_train[,-c(2,10,13,14)], center = center, scale = scale)

gym_test_scaled = gym_test
gym_test_scaled[,-c(2,10,13,14)] <- scale(gym_test[,-c(2,10,13,14)], center = center, scale = scale)


cat("Data loaded and preprocessed")


## FUNCTION DEFINITIONS ##

# Function to plot residuals
# x: predicted values
# y: residuals
gplot.res <- function(x, y, titre = "titre"){
    ggplot(data.frame(x=x, y=y),aes(x,y))+
    geom_point(col = "blue")+#xlim(0, 250)+ylim(-155, 155)+
    ylab("Résidus")+ xlab("Valeurs prédites")+
    ggtitle(titre)+
    geom_hline(yintercept = 0,col="green")
}

# Function to plot ROC curve
# model: model to evaluate
# data: data to evaluate
# title: title of the plot
plot_roc <- function(model, data, title = "ROC curve"){
    pred <- predict(model, data, type = "response")
    roc <- roc(data$Experience_Level, pred)
    auc <- round(auc(roc), 2)
    plot(roc, main = title)
    text(0.8, 0.2, paste("AUC = ", auc), cex = 1.5)
}

# Function that compute R2

# compute_R2 <- function(model, newdata = NA){
#   if newdata == NA{
#     residuals <- predict(model)
#       rss <- sum()
#       tss <- 
# }
#   res.tree.cal.cp_high.test <- predict(tree.reg.cal.cp_high, newdata = gym_test)
#   mse_test_cal_cp_high <- mean((res.tree.cal.cp_high.test - gym_train[,"Calories_Burned"])^2)
#   rss_cal_cp_high <- sum((res.tree.cal.cp_high.test - gym_test[,"Calories_Burned"])^2)
#   tss_cal_cp_high <- sum((gym_test[,"Calories_Burned"] - mean(gym_test[,"Calories_Burned"]))^2)
#   r2_test_cal_cp_high <- 1 - rss_cal_cp_high / tss_cal_cp_high
# }



In [ ]:
summary(gym_train_scaled)
summary(gym_test_scaled)

## CART & Agregation
### Classification Trees

- Fit a classification tree.
- Prune the tree using cross-validation.
- Plot: Decision tree structure.

In [ ]:
# Fit a regression tree model
library(rpart)
library(rpart.plot)
library(MLmetrics)

# Fit a regression tree model for Experience_Level using the training set
tree.reg.exp <- rpart(Experience_Level ~ ., data = gym_train, control=rpart.control(cp=0.0001))

options(repr.plot.width=18, repr.plot.height=12)
# Plot the tree
rpart.plot(tree.reg.exp, extra = 101, type = 3, under = TRUE, cex = 0.8, tweak = 1)

# Convert the target to binary for ROC analysis
# Here we're treating Experience_Level as a binary classification problem
# First, get predicted probabilities for each class
pred_exp_tree_test <- predict(tree.reg.exp, gym_test, type = "prob")
pred_exp_tree_train <- predict(tree.reg.exp, gym_train, type = "prob")

# Get actual classes and convert to numeric matrix for MultiLogLoss
actual_classes <- gym_test$Experience_Level
actual_matrix <- model.matrix(~actual_classes-1)

# Calculate log loss using MultiLogLoss from MLmetrics
logloss_value <- MultiLogLoss(y_pred = pred_exp_tree_test, y_true = actual_matrix)

# Calculate confusion matrix for test set
pred_classes_test <- predict(tree.reg.exp, gym_test, type="class")
conf_matrix_test <- confusionMatrix(pred_classes_test, gym_test$Experience_Level)

cat("Test Set Performance:\n")
cat("- Accuracy:", round(conf_matrix_test$overall["Accuracy"], 4), "\n")
cat("- Log loss:", round(logloss_value, 4), "(lower is better)\n")
cat("- Confusion Matrix:\n")
print(conf_matrix_test$table)
cat("\n")

# Compute the metrics for the gym_train set
# Calculate confusion matrix for training set
pred_classes_train <- predict(tree.reg.exp, gym_train, type="class")
conf_matrix_train <- confusionMatrix(pred_classes_train, gym_train$Experience_Level)

# Get actual classes and convert to numeric matrix for MultiLogLoss
actual_classes_train <- gym_train$Experience_Level
actual_matrix_train <- model.matrix(~actual_classes_train-1)

# Calculate log loss using MultiLogLoss from MLmetrics
logloss_value_train <- MultiLogLoss(y_pred = pred_exp_tree_train, y_true = actual_matrix_train)


cat("Training Set Performance:\n")
cat("- Accuracy:", round(conf_matrix_train$overall["Accuracy"], 4), "\n")
cat("- Log loss:", round(logloss_value_train, 4), "(lower is better)\n")
cat("- Confusion Matrix:\n")
print(conf_matrix_train$table)

# Afficher le nombre de noeuds et de feuilles en français
cat("Nombre de noeuds dans l'arbre : ", length(tree.reg.exp$frame$var), "\n")
cat("Nombre de feuilles dans l'arbre : ", length(tree.reg.exp$frame$var[tree.reg.exp$frame$var == "<leaf>"]), "\n")

**Interprétation :**

L'arbre de régression construit avec un paramètre de complexité faible (`cp = 0.0001`) montre des performances solides mais indique un potentiel sur-apprentissage.

**Performance sur le jeu de test :**
- **Précision :** 87,11 % - Bonne capacité à classer correctement les observations.
- **Log loss :** 0.2528 - Indique des prédictions probabilistes bien calibrées.
- **Matrice de confusion :** Bonne distinction des classes, mais quelques confusions entre les classes 1 et 2.

**Performance sur le jeu d'entraînement :**
- **Précision :** 94,74 % - Ajustement presque parfait aux données d'entraînement.
- **Log loss :** 0.1257 - Confirme un ajustement très précis.

**Conclusion :**
L'écart entre les performances d'entraînement et de test (94,74 % vs 87,11 %) et les log loss (0.1257 vs 0.2528) suggère un sur-apprentissage dû à une complexité excessive. Malgré cela, le modèle capture bien les relations dans les données. Une optimisation du paramètre de complexité (`cp`) pourrait améliorer la généralisation.

### **Sélection du paramètre de complexité (cp) par validation croisée**

Pour éviter le sur-apprentissage et obtenir un arbre de régression optimal, il est important de choisir correctement le paramètre de complexité (`cp`). La méthode classique consiste à utiliser la validation croisée intégrée à l’algorithme `rpart` :

- On ajuste d’abord un arbre avec une valeur de `cp` très faible, ce qui permet à l’arbre de se développer au maximum.
- `rpart` réalise automatiquement une validation croisée (par défaut à 10 plis) et enregistre, pour différentes valeurs de `cp`, l’erreur de validation croisée associée.
- On visualise ensuite l’évolution de l’erreur de validation croisée en fonction de `cp` à l’aide de la fonction `plotcp()`.
- Le `cp` optimal est celui qui minimise l’erreur de validation croisée (`xerror`). On peut aussi appliquer la règle du 1-SE pour sélectionner un arbre plus simple si plusieurs valeurs de `cp` donnent des erreurs similaires.
- Enfin, on élague l’arbre initial avec la valeur optimale de `cp` pour obtenir un modèle plus parcimonieux et mieux généralisant.

Cette démarche permet de sélectionner automatiquement la complexité de l’arbre en fonction des performances sur des données non vues.

In [ ]:
# Ajuster l'arbre avec un cp faible pour explorer toute la complexité
tree.class.exp <- rpart(Experience_Level ~ ., data = gym_train, method = "class", control = rpart.control(cp = 0.0001))

# Visualiser la courbe d'erreur de validation croisée
plotcp(tree.class.exp)

# Extraire la table des cp
cp_table <- tree.class.exp$cptable

# Trouver le cp optimal (celui qui minimise l'erreur de validation croisée)
min_xerror_idx <- which.min(cp_table[,"xerror"])
cp_optimal <- cp_table[min_xerror_idx, "CP"]
cat("Le cp optimal est :", cp_optimal, "\n")

# Élaguer l'arbre avec le cp optimal
tree.class.exp.pruned <- prune(tree.class.exp, cp = cp_optimal)

# Visualiser l'arbre élagué
rpart.plot(tree.class.exp.pruned, extra = 104, type = 3, under = TRUE, cex = 0.8, tweak = 1)

# Prédictions sur le jeu de test
pred_classes_test <- predict(tree.class.exp.pruned, gym_test, type = "class")

# Matrice de confusion
conf_matrix_test <- confusionMatrix(pred_classes_test, gym_test$Experience_Level)

# Afficher le nombre de noeuds et de feuilles
cat("Nombre de noeuds dans l'arbre elague :", length(tree.class.exp.pruned$frame$var), "\n")
cat("Nombre de feuilles dans l'arbre elague :", length(tree.class.exp$frame$var[tree.class.exp$frame$var == "<leaf>"]), "\n")

In [ ]:
# Prédictions sur le jeu de test (classes et probabilités)
pred_classes_test <- predict(tree.class.exp.pruned, gym_test, type = "class")
pred_probs_test <- predict(tree.class.exp.pruned, gym_test, type = "prob")

# Prédictions sur le jeu d'entraînement (classes et probabilités)
pred_classes_train <- predict(tree.class.exp.pruned, gym_train, type = "class")
pred_probs_train <- predict(tree.class.exp.pruned, gym_train, type = "prob")

# Matrices de confusion
conf_matrix_test <- confusionMatrix(pred_classes_test, gym_test$Experience_Level)
conf_matrix_train <- confusionMatrix(pred_classes_train, gym_train$Experience_Level)

# Log loss (nécessite MLmetrics)
actual_matrix_test <- model.matrix(~gym_test$Experience_Level-1)
actual_matrix_train <- model.matrix(~gym_train$Experience_Level-1)
logloss_test <- MultiLogLoss(y_pred = pred_probs_test, y_true = actual_matrix_test)
logloss_train <- MultiLogLoss(y_pred = pred_probs_train, y_true = actual_matrix_train)

# Affichage des résultats
cat("Test Set Performance:\n")
cat("- Accuracy:", round(conf_matrix_test$overall["Accuracy"], 4), "\n")
cat("- Log loss:", round(logloss_test, 4), "(lower is better)\n")
cat("- Confusion Matrix:\n")
print(conf_matrix_test$table)
cat("\n")

cat("Training Set Performance:\n")
cat("- Accuracy:", round(conf_matrix_train$overall["Accuracy"], 4), "\n")
cat("- Log loss:", round(logloss_train, 4), "(lower is better)\n")
cat("- Confusion Matrix:\n")
print(conf_matrix_train$table)

#### Complexité de l’arbre avant et après élagage

- **Premier arbre (cp très faible, arbre complexe) :**
  - Nombre de nœuds dans l’arbre : **37**
  - Nombre de feuilles dans l’arbre : **19**

  Cet arbre très développé s’ajuste fortement aux données d’entraînement, ce qui favorise le sur-apprentissage.

- **Arbre élagué (cp optimal) :**
  - Nombre de nœuds dans l’arbre élagué : **9**
  - Nombre de feuilles dans l’arbre élagué : **19**

  Après élagage, l’arbre conserve le même nombre de feuilles (donc la même capacité de séparation des classes), mais avec beaucoup moins de nœuds internes. Cela signifie que la structure de l’arbre est simplifiée, ce qui améliore la robustesse et la capacité de généralisation du modèle.

#### Résultats avant et après élagage

**Avant élagage (cp très faible, arbre complexe)**

- **Jeu de test :**
  - Accuracy : 0.8711
  - Log loss : 0.2528
  - L’arbre distingue bien les classes, mais il y a des confusions entre les classes 1 et 2.

- **Jeu d’entraînement :**
  - Accuracy : 0.9474
  - Log loss : 0.1257
  - Très bonne performance, mais l’écart avec le test montre un sur-apprentissage.

---

**Après élagage (cp optimal = 0.0044, arbre plus simple)**

- **Jeu de test :**
  - Accuracy : 0.9021 (en hausse)
  - Log loss : 0.1977 (en baisse, donc meilleur)
  - La matrice de confusion montre moins d’erreurs, surtout pour la classe 2.

- **Jeu d’entraînement :**
  - Accuracy : 0.9037 (en baisse, mais plus proche du test)
  - Log loss : 0.1817 (en légère hausse, mais toujours bonne)
  - Les performances sont désormais très proches entre entraînement et test.

---

**Interprétation**

L’élagage avec le cp optimal a permis de réduire le sur-apprentissage : l’écart entre les performances d’entraînement et de test a diminué.  
Le modèle généralise mieux : la précision sur le test augmente, la log loss diminue.  
L’arbre est plus simple et plus robuste : il fait moins d’erreurs sur des données non vues, même si la précision d’entraînement baisse un peu (ce qui est normal et souhaité).

**Conclusion :**  
L’utilisation du cp optimal améliore la capacité de généralisation du modèle, en évitant qu’il ne s’ajuste trop aux particularités du jeu d’entraînement. C’est exactement l’objectif de la validation croisée et de l’élagage.

### Random Forests and Boosting
- Random forests with `mtry` and Brieman criterion
- Regularization with Boosting
- Using Bootstrap
- **plot** feature importance

#### Simple Random Forest

Entraînement du modèle Random Forest

L'objectif des forets aléatoires est de réduire la variance des arbres tout en conservant leur pouvoir prédictif via le bagging, qui est une technique combinant bootstraping et agrégation d'arbres.

In [ ]:
library(randomForest)

In [ ]:
set.seed(24)
# Entraînement du modèle Random Forest
rf_model <- randomForest(Experience_Level ~ ., 
                         data = gym_train,
                         ntree = 500,           # nombre d’arbres
                         mtry = 4,              # nombre de variables testées à chaque split
                         importance = TRUE)     # permet de calculer l’importance des variables

# Résumé du modèle
print(rf_model)

### Interprétation du modèle Random Forest

Le modèle Random Forest a été entraîné pour prédire la variable **Experience_Level** à partir des données d’entraînement.

- **Nombre d’arbres :** 500
- **Variables testées à chaque split (mtry) :** 4

#### Résultats sur le jeu d’entraînement (OOB)

- **Taux d’erreur OOB (Out-Of-Bag) :** 10,53 %
  - Cela signifie que le modèle se trompe en moyenne sur 10,53 % des observations non vues lors de la construction de chaque arbre.
- **Matrice de confusion :**
  - **Classe 1 :** 230 bien classés, 71 mal classés (erreur : 23,6 %)
  - **Classe 2 :** 314 bien classés, 11 mal classés (erreur : 3,4 %)
  - **Classe 3 :** 153 bien classés, 0 mal classés (erreur : 0 %)

#### Interprétation

- Le modèle distingue très bien la classe 3 (aucune erreur) et la classe 2 (faible taux d’erreur).
- Il a plus de difficultés à bien prédire la classe 1 (23,6 % d’erreur).
- Le taux d’erreur global est faible, ce qui indique une bonne capacité de classification sur les données d’entraînement.
- L’utilisation de 500 arbres et la sélection aléatoire de variables à chaque split permettent de limiter le sur-apprentissage et d’améliorer la robustesse du modèle.

Intéressons-nous maintenant à l'évaluation de notre modèle sur nos jeux d'entraînement et de test.


In [ ]:
# Prédictions Random Forest sur le jeu de test (classes et probabilités)
pred_classes_test_rf <- predict(rf_model, gym_test, type = "class")
pred_probs_test_rf <- predict(rf_model, gym_test, type = "prob")

# Prédictions sur le jeu d'entraînement
pred_classes_train_rf <- predict(rf_model, gym_train, type = "class")
pred_probs_train_rf <- predict(rf_model, gym_train, type = "prob")

# Matrices de confusion
conf_matrix_test_rf <- confusionMatrix(pred_classes_test_rf, gym_test$Experience_Level)
conf_matrix_train_rf <- confusionMatrix(pred_classes_train_rf, gym_train$Experience_Level)

# Log loss (nécessite MLmetrics)
actual_matrix_test_rf <- model.matrix(~gym_test$Experience_Level-1)
actual_matrix_train_rf <- model.matrix(~gym_train$Experience_Level-1)
logloss_test_rf <- MultiLogLoss(y_pred = pred_probs_test_rf, y_true = actual_matrix_test_rf)
logloss_train_rf <- MultiLogLoss(y_pred = pred_probs_train_rf, y_true = actual_matrix_train_rf)

# Affichage des résultats
cat("Test Set Performance (Random Forest):\n")
cat("- Accuracy:", round(conf_matrix_test_rf$overall["Accuracy"], 4), "\n")
cat("- Log loss:", round(logloss_test_rf, 4), "(lower is better)\n")
cat("- Confusion Matrix:\n")
print(conf_matrix_test_rf$table)
cat("\n")

cat("Training Set Performance (Random Forest):\n")
cat("- Accuracy:", round(conf_matrix_train_rf$overall["Accuracy"], 4), "\n")
cat("- Log loss:", round(logloss_train_rf, 4), "(lower is better)\n")
cat("- Confusion Matrix:\n")
print(conf_matrix_train_rf$table)

### Interprétation des performances sur le jeu d'entraînement

Le modèle Random Forest affiche une précision (accuracy) de **100%** sur le jeu d'entraînement, ce qui signifie qu’il a correctement classé **l’intégralité des observations** de cet échantillon. Bien que ce résultat puisse sembler excellent, il doit être interprété avec prudence. En effet, cette absence totale d’erreurs sur les données d'entraînement est généralement le signe d’un **surapprentissage** (*overfitting*), c’est-à-dire que le modèle a appris **par cœur les données** plutôt que de généraliser les tendances sous-jacentes.

Ce phénomène est confirmé par la comparaison avec l’estimation de l’erreur **OOB (Out-Of-Bag)**, fournie par le modèle pendant l'entraînement, qui était d’environ **10,78%**. Cette erreur OOB est considérée comme une estimation plus fiable de la performance réelle, car elle est calculée à partir d'observations non utilisées pour construire chaque arbre.

Ainsi, bien que l’accuracy de 100% témoigne d’une forte capacité du modèle à mémoriser les données, elle ne garantit pas une bonne capacité de généralisation. L’évaluation sur un jeu de test indépendant reste indispensable pour valider les performances du modèle.


L'objectif des forêts aléatoires est de réduire la **variance des arbres de décision** tout en conservant leur pouvoir prédictif. Elles reposent sur la technique du **bagging**, qui combine **bootstraping** (tirages aléatoires avec remise) et **agrégation de prédictions** (par vote majoritaire dans le cas de la classification).

Contrairement à un arbre classique, à chaque nœud d’un arbre de la forêt, **un sous-ensemble aléatoire de variables est sélectionné** (de taille `mtry`), et la **meilleure variable parmi ce sous-ensemble** est utilisée pour réaliser la division. Ce mécanisme permet de **décorréler les arbres entre eux**, ce qui renforce la robustesse globale du modèle.

### Paramètres à optimiser :

- **`mtry`** : nombre de variables sélectionnées aléatoirement à chaque split.
    - Dans le cas de la **classification**, la valeur empirique par défaut est `mtry ≈ √p`, où `p` est le nombre total de variables explicatives. Ici, `p = 14`, donc `mtry` vaut par défaut `√p = √14 = 4`.
    - L’optimisation de `mtry` est réalisée via la fonction `tuneRF`, qui cherche le meilleur compromis entre complexité et performance. Cette fonction commence à une valeur initiale (`mtry = 4` ici) et teste des valeurs supérieures et inférieures tant que l’erreur de généralisation mesurée par le taux d’erreur **OOB (Out-Of-Bag)** s’améliore d’au moins 5% (ce seuil est modifiable).

- **`ntree`** : nombre d’arbres dans la forêt.
    - En général, `ntree` varie de 100 à 500. Au-delà, les gains de performance sont marginaux, mais cela peut améliorer la **stabilité** de la prédiction. Dans ce projet, nous utilisons `ntree = 500` pour garantir la convergence des résultats.

Cette combinaison d’aléatoire (dans les données et dans les variables) permet à Random Forest d’offrir un excellent compromis entre **performance** et **généralisation**, même sans réglage excessif d’hyperparamètres.


##### Optimisation du `mtry`

In [ ]:
set.seed(24)

# Optimisation du mtry avec tuneRF
best_mtry <- tuneRF(x = gym_train[ , -which(names(gym_train) == "Experience_Level")],
                    y = gym_train$Experience_Level,
                    stepFactor = 1.5,
                    improve = 0.05,
                    ntreeTry = 100,
                    trace = TRUE,
                    plot = TRUE,
                    doBest = TRUE)

optimal_mtry <- best_mtry$mtry
cat("Optimal mtry value:", optimal_mtry, "\n")

Le paramètre `mtry` a été optimisé automatiquement à l'aide de la fonction `tuneRF()`, qui teste différentes valeurs de `mtry` autour d'une valeur initiale (ici, `√p = 4`), en les multipliant ou divisant par un facteur `stepFactor = 1.5`. L'évaluation de chaque `mtry` est réalisée à l'aide de 100 arbres (`ntreeTry = 100`) afin d'obtenir une estimation rapide mais fiable de l'erreur OOB. 

Le processus de tuning s'arrête dès qu'aucune amélioration significative (au moins `5%`) de l'erreur OOB n’est observée (`improve = 0.05`). L’option `doBest = TRUE` permet enfin de réentraîner automatiquement un modèle avec la meilleure valeur trouvée, ce qui facilite l'intégration du tuning dans le workflow.

Cette procédure a confirmé que `mtry = 4` (valeur théorique par défaut pour la classification avec 14 variables explicatives) offrait les meilleures performances, sans amélioration notable au-delà.


In [ ]:
# Prédictions Random Forest optimisé (best_mtry) sur test et train
pred_classes_test_best <- predict(best_mtry, gym_test, type = "class")
pred_probs_test_best <- predict(best_mtry, gym_test, type = "prob")

pred_classes_train_best <- predict(best_mtry, gym_train, type = "class")
pred_probs_train_best <- predict(best_mtry, gym_train, type = "prob")

# Matrices de confusion
conf_matrix_test_best <- confusionMatrix(pred_classes_test_best, gym_test$Experience_Level)
conf_matrix_train_best <- confusionMatrix(pred_classes_train_best, gym_train$Experience_Level)

# Log loss (MultiLogLoss)
actual_matrix_test_best <- model.matrix(~gym_test$Experience_Level-1)
actual_matrix_train_best <- model.matrix(~gym_train$Experience_Level-1)
logloss_test_best <- MultiLogLoss(y_pred = pred_probs_test_best, y_true = actual_matrix_test_best)
logloss_train_best <- MultiLogLoss(y_pred = pred_probs_train_best, y_true = actual_matrix_train_best)

# Affichage des résultats
cat("Test Set Performance (Random Forest optimisé):\n")
cat("- Accuracy:", round(conf_matrix_test_best$overall["Accuracy"], 4), "\n")
cat("- Log loss:", round(logloss_test_best, 4), "(lower is better)\n")
cat("- Confusion Matrix:\n")
print(conf_matrix_test_best$table)
cat("\n")

cat("Training Set Performance (Random Forest optimisé):\n")
cat("- Accuracy:", round(conf_matrix_train_best$overall["Accuracy"], 4), "\n")
cat("- Log loss:", round(logloss_train_best, 4), "(lower is better)\n")
cat("- Confusion Matrix:\n")
print(conf_matrix_train_best$table)

In [ ]:
# Importance des variables
library(ggRandomForests)

#Plot importance
varImpPlot(rf_model, main = "Importance des variables (Random Forest)")

L'importance des variables a été évaluée à l’aide des mesures intégrées à l’algorithme Random Forest. Deux métriques complémentaires ont été analysées :

- **Mean Decrease in Accuracy** : elle mesure la perte de précision du modèle lorsque les valeurs d’une variable sont aléatoirement permutées. Cette mesure est considérée comme la plus fiable car elle reflète directement l’impact de la variable sur la performance globale du modèle.
- **Mean Decrease in Gini** : elle correspond à la réduction moyenne de l’impureté des nœuds (indice de Gini) induite par l’utilisation d’une variable pour diviser les données dans les arbres. Bien qu’efficace, cette métrique peut être biaisée en faveur des variables numériques continues.

Le graphique montre que des variables telles que `Session_Duration..hours.`, `Workout_Frequency` ou encore `Fat_Percentage` sont parmi les plus influentes dans la prédiction du niveau d’expérience. Ces résultats sont cohérents avec l’analyse exploratoire, qui avait déjà mis en évidence des corrélations fortes entre ces variables et la variable cible `Experience_Level`.

Cette observation est tout à fait cohérente : les profils les plus expérimentés sont aussi ceux qui cumulent une fréquence d'entraînement élevée, un volume d’exercice important et un taux de masse grasse plus bas, autant d’indicateurs caractéristiques d’une pratique sportive régulière et avancée.

L’analyse de l’importance des variables permet non seulement d’interpréter le modèle, mais aussi d’identifier les facteurs les plus déterminants dans la progression des adhérents d'une salle de sport.


#### Boosting
##### Avec la librairie `xgboost`


#### 1. Entraînement d'un premier modèle baseline

Nous avons d'abord construit un modèle de base en spécifiant les paramètres suivants :
- `objective = "multi:softprob"` : configuration adaptée à une classification multi-classes, permettant d'obtenir les probabilités associées à chaque classe.
- `num_class = 3` : car la variable cible comporte trois niveaux.
- `eval_metric = "mlogloss"` : la fonction de perte choisie est le log loss, adaptée aux probabilités.
- `eta = 0.1` : taux d'apprentissage modéré pour stabiliser l'entraînement.
- `max_depth = 6` : profondeur maximale des arbres.
- `subsample = 0.8` et `colsample_bytree = 0.8` : échantillonnage partiel des données et des variables pour limiter le surapprentissage.

Ce modèle initial a été entraîné avec 300 arbres (`nrounds = 300`).  
Les prédictions ont ensuite été obtenues sous forme de probabilités pour chaque classe.  
La classe prédite correspond à celle ayant la probabilité la plus élevée.

Une évaluation préliminaire par **matrice de confusion** sur le jeu de test a permis d'obtenir une **accuracy de 85.57%**.

---

In [ ]:
library(xgboost)
library(caret)

# Créer les matrices d'entraînement
X_train <- model.matrix(Experience_Level ~ . -1, data = gym_train)
y_train <- as.numeric(gym_train$Experience_Level) - 1  # XGBoost demande des labels 0, 1, 2

X_test <- model.matrix(Experience_Level ~ . -1, data = gym_test)
y_test <- as.numeric(gym_test$Experience_Level) - 1

# Transformer en objets DMatrix pour XGBoost
dtrain <- xgb.DMatrix(data = X_train, label = y_train)
dtest <- xgb.DMatrix(data = X_test, label = y_test)


In [ ]:
# Définir les paramètres
params <- list(
  objective = "multi:softprob",   # pour multi-classes avec probabilités
  num_class = 3,                  # 3 classes ici
  eval_metric = "mlogloss"         # log loss pour la stabilité
)

# Entraîner
set.seed(24)
xgb_model <- xgb.train(
  params = params,
  data = dtrain,
  nrounds = 300,          # 300 arbres
  eta = 0.1,              # learning rate correct
  max_depth = 6,          # arbres raisonnablement profonds
  subsample = 0.8,        # sous-échantillonnage pour éviter overfitting
  colsample_bytree = 0.8, # sous-échantillonnage des colonnes
  verbose = 0
)


In [ ]:
# Prédictions : probabilités
pred_probs <- predict(xgb_model, newdata = dtest)

# Reformater en matrice : chaque ligne correspond à un individu
pred_matrix <- matrix(pred_probs, ncol = 3, byrow = TRUE)

# Prendre la classe la plus probable
pred_classes <- max.col(pred_matrix) - 1  # car max.col commence à 1

# Comparer aux vraies classes
confusionMatrix(as.factor(pred_classes), as.factor(y_test))

#### 2. Optimisation des hyperparamètres par validation croisée

Afin d'améliorer ce modèle, nous avons mis en place une **recherche sur grille (grid search)** couplée à une **validation croisée 5 folds**.  
La grille explorait les paramètres suivants :
- `nrounds` : 300 et 500
- `max_depth` : 4, 6, 8
- `eta` : 0.05 et 0.1
- `gamma` : 0 et 1
- `colsample_bytree` : 0.7 et 1
- `min_child_weight` : 1 et 5
- `subsample` : 0.7 et 1

Le critère d'optimisation était la **précision (`Accuracy`)** sur les données de validation.

Cette recherche a permis d'identifier la meilleure combinaison d'hyperparamètres.

---

In [ ]:
# Grille d'hyperparamètres à explorer
grid <- expand.grid(
  nrounds = c(300, 500),
  max_depth = c(4, 6, 8),
  eta = c(0.05, 0.1),
  gamma = c(0, 1),
  colsample_bytree = c(0.7, 1),
  min_child_weight = c(1, 5),
  subsample = c(0.7, 1)
)

# 5-fold CV
ctrl <- trainControl(
  method = "cv",
  number = 5,
  verboseIter = TRUE,
  allowParallel = TRUE
)

set.seed(24)
xgb_tuned_full <- train(
  Experience_Level ~ .,
  data = gym_train, # Utiliser gym_train au lieu de train_data
  method = "xgbTree",
  trControl = ctrl,
  tuneGrid = grid,
  metric = "Accuracy",
  verbose = 0 # Mettre à 1 ou TRUE pour voir la progression si désiré
)


#### 3. Entraînement du modèle final

Un modèle final a ensuite été réentraîné sur l'intégralité du jeu `gym_train` en utilisant les hyperparamètres optimaux trouvés.  
Ce modèle utilise :
- Le meilleur `nrounds`
- Le meilleur `max_depth`
- Le meilleur `eta`, `gamma`, `subsample`, `colsample_bytree`, `min_child_weight`

---

In [ ]:
# Extraire les meilleurs paramètres trouvés
best_params <- xgb_tuned_full$bestTune

# Créer une matrice DMatrix pour tout gym_train
dtrain <- xgb.DMatrix(data = X_train, label = y_train)

# Réentraîner avec les meilleurs hyperparamètres
set.seed(24)
xgb_final_model <- xgboost(
  data = dtrain,
  objective = "multi:softprob",
  num_class = 3,
  eval_metric = "mlogloss",
  nrounds = best_params$nrounds,
  max_depth = best_params$max_depth,
  eta = best_params$eta,
  gamma = best_params$gamma,
  colsample_bytree = best_params$colsample_bytree,
  min_child_weight = best_params$min_child_weight,
  subsample = best_params$subsample,
  verbose = 0
)

#### 4. Évaluation finale

Les prédictions finales sur le jeu de test `gym_test` ont été réalisées en prenant la classe associée à la probabilité la plus élevée.

L'évaluation par **matrice de confusion** a permis d'obtenir une **accuracy finale** (précision) correspondant à la performance maximale atteinte par le modèle optimisé.

---

In [ ]:
# Prédictions finales XGBoost sur le jeu de test (probabilités)
pred_probs_final <- predict(xgb_final_model, newdata = dtest)
pred_matrix_final <- matrix(pred_probs_final, ncol = 3, byrow = TRUE)

# Classe prédite = colonne avec la probabilité max (commence à 1)
pred_classes_final <- max.col(pred_matrix_final)
# Adapter les niveaux pour correspondre à gym_test$Experience_Level
pred_classes_final_factor <- factor(pred_classes_final, levels = 1:3, labels = levels(gym_test$Experience_Level))

# Calcul MultiLogLoss pour le test
actual_matrix_test_xgb <- model.matrix(~gym_test$Experience_Level-1)
logloss_test_xgb <- MultiLogLoss(y_pred = pred_matrix_final, y_true = actual_matrix_test_xgb)

# Accuracy et matrice de confusion (test)
conf_matrix_test_xgb <- confusionMatrix(pred_classes_final_factor, gym_test$Experience_Level)
accuracy_test_xgb <- conf_matrix_test_xgb$overall["Accuracy"]

# Prédictions sur le jeu d'entraînement (probabilités)
pred_probs_train_final <- predict(xgb_final_model, newdata = dtrain)
pred_matrix_train_final <- matrix(pred_probs_train_final, ncol = 3, byrow = TRUE)
pred_classes_train_final <- max.col(pred_matrix_train_final)
pred_classes_train_final_factor <- factor(pred_classes_train_final, levels = 1:3, labels = levels(gym_train$Experience_Level))

actual_matrix_train_xgb <- model.matrix(~gym_train$Experience_Level-1)
logloss_train_xgb <- MultiLogLoss(y_pred = pred_matrix_train_final, y_true = actual_matrix_train_xgb)

# Accuracy et matrice de confusion (train)
conf_matrix_train_xgb <- confusionMatrix(pred_classes_train_final_factor, gym_train$Experience_Level)
accuracy_train_xgb <- conf_matrix_train_xgb$overall["Accuracy"]

# Affichage des résultats
cat("MultiLogLoss XGBoost (test):", round(logloss_test_xgb, 4), "\n")
cat("Accuracy XGBoost (test):", round(accuracy_test_xgb, 4), "\n")
cat("Matrice de confusion (test):\n")
print(conf_matrix_test_xgb$table)
cat("\n")

cat("MultiLogLoss XGBoost (train):", round(logloss_train_xgb, 4), "\n")
cat("Accuracy XGBoost (train):", round(accuracy_train_xgb, 4), "\n")
cat("Matrice de confusion (train):\n")
print(conf_matrix_train_xgb$table)

### Comparaison des performances des modèles

| Modèle         | Accuracy | Log Loss |
|----------------|----------|----------|
| CART           | 0.902    | 0.198    |
| Random Forest  | 0.892    | 0.246    |
| XGBoost        | 0.881    | 0.224    |

### Synthèse finale : Comparaison de CART, Random Forest et XGBoost

Trois modèles d'apprentissage supervisé ont été comparés pour prédire le niveau d'expérience (`Experience_Level`) :

- **CART** (Classification and Regression Trees) affiche la meilleure performance globale, avec une **accuracy** de ~90,2% et la **log loss** la plus faible (0.198). Cela indique non seulement une bonne capacité de classification, mais aussi des probabilités de prédiction bien calibrées.
  
- **Random Forest** obtient une accuracy de ~89,2% et une log loss de 0.246. Le modèle reste robuste et performant, mais ses probabilités sont un peu moins précises que celles du CART, comme l'indique la log loss plus élevée.

- **XGBoost** atteint une accuracy de ~88,1% et une log loss de 0.224. Malgré un tuning approfondi, il reste légèrement en retrait sur ce jeu de données, mais ses probabilités sont mieux calibrées que celles du Random Forest.

En conclusion, bien que Random Forest et XGBoost soient réputés pour leur puissance en modélisation avancée, le modèle CART, plus simple, s'est révélé le plus adapté à ce jeu de données, combinant **performance élevée**, **interprétabilité** et **prédictions probabilistes fiables** (log loss minimale).  
Le choix final du modèle dépendra donc du compromis souhaité entre performance pure, qualité des probabilités et simplicité d'interprétation.



In [ ]:
# Ensure pROC is loaded (if not already)
# library(pROC)

# Ensure Experience_Level is a factor
gym_test$Experience_Level <- as.factor(gym_test$Experience_Level)
class_levels <- levels(gym_test$Experience_Level)

# Define colors for each model (CART, Random Forest, XGBoost)
model_colors <- c("blue", "red", "green") 
model_names <- c("CART", "Random Forest", "XGBoost")

# Loop through each class to create a separate plot
for (i in 1:length(class_levels)) {
  current_class <- class_levels[i]
  
  legend_labels_for_class <- c()
  
  # Create binary response: 1 if current_class, 0 otherwise
  # This response is the same for all models for this specific class
  response_binary <- ifelse(gym_test$Experience_Level == current_class, 1, 0)
  
  # 1. CART Model
  # Probabilities for the current class from CART predictions
  # Assuming pred_probs_test columns are ordered according to class_levels
  prob_cart_current_class <- pred_probs_test[, i]
  roc_obj_cart <- roc(response = response_binary, predictor = prob_cart_current_class, quiet = TRUE)
  
  # Plot ROC curve for CART (this will be the first curve on the new plot)
  # Add title directly to the plot call
  plot(roc_obj_cart, col = model_colors[1], legacy.axes = TRUE, print.auc = FALSE,
       main = paste("ROC Curves for Class '", current_class, "' vs Rest", sep=""),
       sub = "Comparison of CART, Random Forest, and XGBoost")
  legend_labels_for_class <- c(legend_labels_for_class, 
                               paste(model_names[1], " (AUC = ", round(auc(roc_obj_cart), 3), ")", sep=""))
  
  # 2. Random Forest Model
  # Probabilities for the current class from Random Forest predictions
  # Assuming pred_probs_test_rf columns are ordered according to class_levels
  prob_rf_current_class <- pred_probs_test_rf[, i]
  roc_obj_rf <- roc(response = response_binary, predictor = prob_rf_current_class, quiet = TRUE)
  
  # Plot ROC curve for Random Forest (add to the existing plot)
  plot(roc_obj_rf, col = model_colors[2], add = TRUE, print.auc = FALSE)
  legend_labels_for_class <- c(legend_labels_for_class, 
                               paste(model_names[2], " (AUC = ", round(auc(roc_obj_rf), 3), ")", sep=""))
  
  # 3. XGBoost Model
  # Probabilities for the current class from XGBoost predictions
  # Assuming pred_matrix_final columns are ordered according to class_levels
  # (e.g., col 1 for class_levels[1], col 2 for class_levels[2], etc.)
  prob_xgb_current_class <- pred_matrix_final[, i] 
  roc_obj_xgb <- roc(response = response_binary, predictor = prob_xgb_current_class, quiet = TRUE)
  
  # Plot ROC curve for XGBoost (add to the existing plot)
  plot(roc_obj_xgb, col = model_colors[3], add = TRUE, print.auc = FALSE)
  legend_labels_for_class <- c(legend_labels_for_class, 
                               paste(model_names[3], " (AUC = ", round(auc(roc_obj_xgb), 3), ")", sep=""))
  
  # Add legend to the current plot
  legend("bottomright", legend = legend_labels_for_class, col = model_colors, lwd = 2, cex = 0.8)
}


## Analyse des courbes ROC pour les classes 1, 2 et 3

### Classe 1
- **AUC (CART = 0.954, Random Forest = 0.941, XGBoost = 0.946)** :
  - Le modèle **CART** est légèrement supérieur à **XGBoost** et **Random Forest** pour la classe `1`.
  - Les trois modèles montrent une bonne capacité de discrimination pour cette classe, avec des AUC proches de 1.
- **Conclusion** : CART est le modèle le plus performant pour la classe `1`, suivi de près par XGBoost.

---

### Classe 2
- **AUC (CART = 0.955, Random Forest = 0.943, XGBoost = 0.948)** :
  - CART reste légèrement meilleur que XGBoost et Random Forest pour la classe `2`.
  - Les performances des trois modèles sont similaires, avec des AUC très proches.
- **Conclusion** : CART est encore une fois le modèle le plus performant, mais XGBoost est presque équivalent.

---

### Classe 3
- **AUC (CART = 1, Random Forest = 1, XGBoost = 1)** :
  - Tous les modèles atteignent une AUC parfaite de `1` pour la classe `3`.
  - Cela signifie que les trois modèles discriminent parfaitement la classe `3` des autres classes.
- **Conclusion** : Les trois modèles sont équivalents pour la classe `3`.

---

### Synthèse globale
1. **Classe 1 et Classe 2** :
   - CART est légèrement meilleur que XGBoost et Random Forest pour ces classes, mais les différences sont minimes.
   - Les trois modèles montrent une excellente capacité de discrimination avec des AUC supérieures à `0.94`.

2. **Classe 3** :
   - Tous les modèles atteignent une performance parfaite (AUC = 1), ce qui indique que cette classe est très bien séparée des autres.

3. **Recommandation** :
   - CART est globalement le modèle le plus performant pour ce problème, avec des AUC légèrement supérieures pour les classes `1` et `2`.
   - XGBoost est une alternative compétitive, particulièrement pour des problèmes où une optimisation fine est nécessaire.
   - Random Forest reste robuste et performant, mais légèrement en retrait par rapport aux deux autres modèles.